Experiments with cython (perf x100)

In [ ]:
import math
import time

import pandas as pd
import numpy as np
import plotly as plt

pd.options.plotting.backend = "plotly"

import sys, os, os.path

sys.path.append(os.path.expanduser("../src"))

from generate_common import custom_ray_init, cache_load

In [ ]:
custom_ray_init({"--log-level": "INFO"})

In [ ]:
speaker_name = "Genelec 8341A"
speaker_origin = "ASR"
speaker_version = "asr-vertical"
# speaker_name = "BIC America FH6-LCR Center"
# speaker_origin = "ASR"
# speaker_version = "asr-vertical"
df_speaker = cache_load({"speaker_name": speaker_name, "origin": speaker_origin}, False)

In [ ]:
spl_h = df_speaker[speaker_name][speaker_origin][speaker_version]["SPL Horizontal_unmelted"]
spl_v = df_speaker[speaker_name][speaker_origin][speaker_version]["SPL Vertical_unmelted"]

from spinorama.compute_cea2034 import compute_cea2034, estimated_inroom_hv

%timeit compute_cea2034(spl_h, spl_v)
%timeit estimated_inroom_hv(spl_h, spl_v)

python_spin = compute_cea2034(spl_h, spl_v)
python_pir = estimated_inroom_hv(spl_h, spl_v)

freq = spl_h.Freq
spl_h = spl_h.drop("Freq", axis=1)
spl_v = spl_v.drop("Freq", axis=1)
spl = np.concatenate((spl_h.T.to_numpy(), spl_v.T.to_numpy()), axis=0)

In [ ]:
%load_ext Cython

In [ ]:
def compute_area_q(alpha_d: float, beta_d: float) -> float:
    """Compute the area of the sphere between 4 lines at alpha and beta angles"""
    alpha = alpha_d * 2 * math.pi / 360
    beta = beta_d * 2 * math.pi / 360
    gamma = math.acos(math.cos(alpha) * math.cos(beta))
    A = math.atan(math.sin(beta) / math.tan(alpha))
    B = math.atan(math.sin(alpha) / math.tan(beta))
    C = math.acos(-math.cos(A) * math.cos(B) + math.sin(A) * math.sin(B) * math.cos(gamma))
    S = 4 * C - 2 * math.pi
    # print('gamma {} A {} B {} C {} S {}'.format(
    #    gamma*360/2/math.pi, A*360/2/math.pi, B*360/2/math.pi, C*360/2/math.pi, S))
    return S


def compute_weigths() -> list[float]:
    """Compute the weigths from the CEA2034 standards"""
    angles = [i * 10 + 5 for i in range(0, 9)] + [90]
    weigth_angles = [compute_area_q(i, i) for i in angles]
    # weigths are the delta between 2 consecutive areas
    weigths = [weigth_angles[0]] + [
        weigth_angles[i] - weigth_angles[i - 1] for i in range(1, len(weigth_angles))
    ]
    weigths[9] *= 2.0
    return weigths


std_weigths = compute_weigths()

sp_weigths = {
    "On Axis": std_weigths[0],
    "180°": std_weigths[0],
    "-180°": std_weigths[0],
    #
    "10°": std_weigths[1],
    "170°": std_weigths[1],
    "-170°": std_weigths[1],
    "-10°": std_weigths[1],
    #
    "20°": std_weigths[2],
    "160°": std_weigths[2],
    "-160°": std_weigths[2],
    "-20°": std_weigths[2],
    #
    "30°": std_weigths[3],
    "150°": std_weigths[3],
    "-150°": std_weigths[3],
    "-30°": std_weigths[3],
    #
    "40°": std_weigths[4],
    "140°": std_weigths[4],
    "-140°": std_weigths[4],
    "-40°": std_weigths[4],
    #
    "50°": std_weigths[5],
    "130°": std_weigths[5],
    "-130°": std_weigths[5],
    "-50°": std_weigths[5],
    #
    "60°": std_weigths[6],
    "120°": std_weigths[6],
    "-120°": std_weigths[6],
    "-60°": std_weigths[6],
    #
    "70°": std_weigths[7],
    "110°": std_weigths[7],
    "-110°": std_weigths[7],
    "-70°": std_weigths[7],
    #
    "80°": std_weigths[8],
    "100°": std_weigths[8],
    "-100°": std_weigths[8],
    "-80°": std_weigths[8],
    #
    "90°": std_weigths[9],
    "-90°": std_weigths[9],
}


def octave(N: int) -> list[tuple[float, float, float]]:
    """compute 1/N octave band

    N: >=2 when N increases, bands are narrower
    """
    # why 1290 and not 1000?
    reference = 1290.0
    p = pow(2, 1 / N)
    p_band = pow(2, 1 / (2 * N))
    o_iter = int((N * 10 + 1) / 2)
    center = (
        [reference / p**i for i in range(o_iter, 0, -1)]
        + [reference]
        + [reference * p**i for i in range(1, o_iter + 1, 1)]
    )
    return [(c / p_band, c, c * p_band) for c in center]


intervals = [
    (np.searchsorted(freq, omin, side="right"), np.searchsorted(freq, omax, side="left"))
    for omin, ocenter, omax in octave(2)
    if 100 <= ocenter <= 12000
]

In [ ]:
spl_keys = [f"H{k}" for k in spl_h] + [f"V{k}" for k in spl_v]
weigths = [sp_weigths[k] for k in spl_h] + [sp_weigths[k] for k in spl_v]


def build_index(spl_key, h_key, v_key):
    idx = [spl_key.index(f"H{k}") for k in h_key] + [spl_key.index(f"V{k}") for k in v_key]
    return idx


idx_lw = build_index(
    spl_keys, ["10°", "20°", "30°", "-10°", "-20°", "-30°"], ["On Axis", "10°", "-10°"]
)
idx_er = build_index(
    spl_keys,
    [
        "On Axis",
        "10°",
        "20°",
        "30°",
        "40°",
        "50°",
        "60°",
        "70°",
        "80°",
        "90°",
        "100°",
        "110°",
        "120°",
        "130°",
        "140°",
        "150°",
        "160°",
        "170°",
        "-10°",
        "-20°",
        "-30°",
        "-40°",
        "-50°",
        "-60°",
        "-70°",
        "-80°",
        "-90°",
        "100°",
        "110°",
        "120°",
        "130°",
        "140°",
        "150°",
        "160°",
        "170°",
        "180°",
    ],
    ["On Axis", "-20°", "-30°", "-40°", "40°", "50°", "60°"],
)
idx_floor_bounce = build_index(spl_keys, [], ["-20°", "-30°", "-40°"])
idx_ceiling_bounce = build_index(spl_keys, [], ["40°", "50°", "60°"])
idx_front_wall_bounce = build_index(
    spl_keys, ["On Axis", "-10°", "-20°", "-30°", "10°", "20°", "30°"], []
)
idx_side_wall_bounce = build_index(
    spl_keys, ["-40°", "-50°", "-60°", "-70°", "-80°", "40°", "50°", "60°", "70°", "80°"], []
)
idx_rear_wall_bounce = build_index(
    spl_keys,
    [
        "-170°",
        "-160°",
        "-150°",
        "-140°",
        "-130°",
        "-120°",
        "-110°",
        "-100°",
        "-90°",
        "90°",
        "100°",
        "110°",
        "120°",
        "130°",
        "140°",
        "150°",
        "160°",
        "170°",
        "180°",
    ],
    [],
)
idx_tvr = build_index(spl_keys, [], ["On Axis", "-20°", "-30°", "-40°", "40°", "50°", "60°"])
idx_fr = build_index(spl_keys, [], ["On Axis", "-20°", "-30°", "-40°"])
idx_cr = build_index(spl_keys, [], ["On Axis", "40°", "50°", "60°"])
idx_front = build_index(spl_keys, ["On Axis", "10°", "20°", "30°", "-10°", "-20°", "-30°"], [])
idx_side = build_index(
    spl_keys, ["40°", "50°", "60°", "70°", "80°", "-40°", "-50°", "-60°", "-70°", "-80°"], []
)
idx_rear = build_index(
    spl_keys,
    [
        "90°",
        "100°",
        "110°",
        "120°",
        "130°",
        "140°",
        "150°",
        "160°",
        "170°",
        "-90°",
        "-100°",
        "-110°",
        "-120°",
        "-130°",
        "-140°",
        "-150°",
        "-160°",
        "-170°",
        "180°",
    ],
    [],
)
idx_thr = build_index(
    spl_keys,
    [
        "On Axis",
        "10°",
        "20°",
        "30°",
        "40°",
        "50°",
        "60°",
        "70°",
        "80°",
        "90°",
        "100°",
        "110°",
        "120°",
        "130°",
        "140°",
        "150°",
        "160°",
        "170°",
        "-10°",
        "-20°",
        "-30°",
        "-40°",
        "-50°",
        "-60°",
        "-70°",
        "-80°",
        "-90°",
        "-100°",
        "-110°",
        "-120°",
        "-130°",
        "-140°",
        "-150°",
        "-160°",
        "-170°",
        "180°",
    ],
    [],
)
idx_sp = build_index(
    spl_keys,
    [
        "On Axis",
        "10°",
        "20°",
        "30°",
        "40°",
        "50°",
        "60°",
        "70°",
        "80°",
        "90°",
        "100°",
        "110°",
        "120°",
        "130°",
        "140°",
        "150°",
        "160°",
        "170°",
        "-10°",
        "-20°",
        "-30°",
        "-40°",
        "-50°",
        "-60°",
        "-70°",
        "-80°",
        "-90°",
        "-100°",
        "-110°",
        "-120°",
        "-130°",
        "-140°",
        "-150°",
        "-160°",
        "-170°",
        "180°",
    ],
    [
        "10°",
        "20°",
        "30°",
        "40°",
        "50°",
        "60°",
        "70°",
        "80°",
        "90°",
        "100°",
        "110°",
        "120°",
        "130°",
        "140°",
        "150°",
        "160°",
        "170°",
        "-10°",
        "-20°",
        "-30°",
        "-40°",
        "-50°",
        "-60°",
        "-70°",
        "-80°",
        "-90°",
        "-100°",
        "-110°",
        "-120°",
        "-130°",
        "-140°",
        "-150°",
        "-160°",
        "-170°",
    ],
)
idx_cea2034 = [
    idx_lw,  # 0
    idx_er,  # 1
    idx_floor_bounce,  # 2
    idx_ceiling_bounce,  # 3
    idx_front_wall_bounce,  # 4
    idx_side_wall_bounce,  # 5
    idx_rear_wall_bounce,  # 6
    idx_tvr,  # 7
    idx_fr,  # 8
    idx_cr,  # 9
    idx_front,  # 10
    idx_side,  # 11
    idx_rear,  # 12
    idx_thr,  # 13
    idx_sp,  # 14
]

In [ ]:
%%cython
import math
cimport numpy as np
import numpy as np
from more_itertools import consecutive_groups
from scipy.stats import linregress

np.import_array()

cdef double[:] spl2pressure(const double[:] spl):
    return np.power(10, np.divide(np.subtract(spl,105.0),20))

cdef double[:] pressure2spl(const double[:] pressure):
    return np.add(np.multiply(np.log10(pressure), 20), 105)

cdef double[:,:] spl2pressure2(const double[:,:] spl):
    cdef p2 = np.zeros_like(spl)
    cdef Py_ssize_t i
    for i in range(spl.shape[0]):
        # spl 2 pressure and then square it
        p2[i] = np.square(spl2pressure(spl[i]))
    return p2


cdef double[:] apply_rms(const double[:,:] p2, idx):
    cdef Py_ssize_t len_idx = len(idx)
    cdef double[:] rms = np.zeros(p2.shape[1])
    cdef Py_ssize_t i
    for i in range(len_idx):
        rms = np.add(rms, p2[idx[i]])
    return pressure2spl(np.sqrt(np.divide(rms, len_idx)))


cdef double[:] apply_weigthed_rms(const double[:,:] p2, idx, const double[:] weigths):
    cdef Py_ssize_t len_idx = len(idx)
    cdef double[:] rms = np.zeros(p2.shape[1])
    cdef double sum_weigths = 0.0
    cdef Py_ssize_t i
    for i in range(len_idx):
        rms = np.add(rms, np.multiply(p2[idx[i]],weigths[idx[i]]))
        sum_weigths += weigths[idx[i]]
    return pressure2spl(np.sqrt(np.divide(rms, sum_weigths)))

cpdef c_cea2034(const double[:,:] spl, idx, const double[:] weigths):
    cdef Py_ssize_t len_spl = len(spl[0])
    cdef pressure2 = spl2pressure2(spl)
    cdef cea2034 = np.zeros([len(idx)+1, len_spl])
    cdef Py_ssize_t idx_lw = 0
    cdef Py_ssize_t idx_er = 1
    cdef Py_ssize_t idx_sp = len(idx)-1
    cdef Py_ssize_t idx_pir = idx_sp+1
    cdef Py_ssize_t i 
    # compute all curves from cea2034
    for i in range(idx_sp):
        cea2034[i] = apply_rms(pressure2, idx[i])
    # ER is computed differently
    cdef double[:] er = np.zeros([len_spl])
    for i in range(2, 7):
        er = np.add(er, np.square(spl2pressure(cea2034[i])))
    cea2034[idx_er] = pressure2spl(np.sqrt(np.divide(er, 5)))
    # SP is weighted rms
    cea2034[idx_sp] = apply_weigthed_rms(pressure2, idx[idx_sp], weigths)
    # EIR
    cea2034[idx_pir] = pressure2spl(
        np.multiply(0.12, spl2pressure(cea2034[idx_lw]))+
        np.multiply(0.44, spl2pressure(cea2034[idx_er]))+
        np.multiply(0.44, spl2pressure(cea2034[idx_sp]))
    )
    return cea2034

cdef double LFX_DEFAULT = math.log10(300)

cdef double mad(const double[:] spl, Py_ssize_t imin, Py_ssize_t imax):
    return np.mean(np.abs(np.subtract(spl[imin:imax], np.mean(spl[imin:imax]))))

cpdef double c_nbd(const double[:] freq, interval, const double[:] spl):
    return np.nanmean([mad(spl, imin,imax) for imin, imax in interval])

cdef first(l):
    return l[0]

cpdef double c_lfx(const double[:] freq, const double[:] lw, const double[:] sp):
    cdef lw_min = np.searchsorted(freq, 300, side='right')
    cdef lw_max = np.searchsorted(freq, 10000, side='left')
    cdef double lw_ref = np.mean(lw[lw_min:lw_max])-6
    cdef lfx_range = [(i, f) for i, f in enumerate(freq[:lw_min]) if sp[i]<=lw_ref]
    if len(lfx_range) == 0:
        return math.log10(freq[0])
    
    lfx_grouped = consecutive_groups(lfx_range, first)
    lfx_list = list(next(lfx_grouped))
    if len(lfx_list) <= 1:
        return LFX_DEFAULT
    return math.log10(lfx_list[-1][1])

cpdef double c_sm(const double[:] freq, const double[:] spl):
    cdef f_min = np.searchsorted(freq, 100, side='right')
    cdef f_max = np.searchsorted(freq, 16000, side='left')
    cdef log_freq = np.log(freq[f_min:f_max])
    _, _, r_value, _, _ = linregress(log_freq, spl[f_min:f_max])
    return r_value*r_value

cpdef c_score(
    const double[:] freq, 
    intervals, 
    const double[:] on, 
    const double[:] lw, 
    const double[:] sp, 
    const double[:] pir
):
    cdef nbd_on = c_nbd(freq, intervals, on)
    cdef nbd_pir = c_nbd(freq, intervals, pir)
    cdef sm_pir = c_sm(freq, pir)
    cdef lf_x = c_lfx(freq, lw, sp)
    cdef score = 12.69 - 2.49 * nbd_on - 2.99 * nbd_pir - 4.31 * lf_x + 2.32 * sm_pir
    return {'nbd_on': nbd_on, 'nbd_pir': nbd_pir, 'lfx': lf_x, 'sm_pir': sm_pir, 'pref_score': score}

cpdef c_score_peq(
    const double[:] freq, 
    idx, 
    intervals, 
    const double[:] weigths, 
    const double[:, :] spl_h, 
    const double[:, :] spl_v, 
    const double[:] peq
):
    cdef spl_h_peq = np.zeros_like(spl_h)
    cdef spl_v_peq = np.zeros_like(spl_v)
    for i in range(spl_h.shape[0]):
        spl_h_peq[i] = np.add(spl_h[i], peq)
        spl_v_peq[i] = np.add(spl_v[i], peq)
    cdef spl = np.concatenate((spl_h_peq, spl_v_peq), axis=0)
    cdef spin = c_cea2034(spl, idx, weigths)
    return spin, c_score(freq, intervals, spl_h_peq[17], spin[0], spin[-2], spin[-1])

In [ ]:
%timeit cea2034(spl, idx_cea2034, np.array(weigths))
spin = c_cea2034(spl, idx_cea2034, np.array(weigths))

In [ ]:
fig = pd.DataFrame(
    {
        "Freq": freq,
        "on": spl_h["On Axis"],
        "lw": spin[0],
        "er": spin[1],
        "sp": spin[-2],
        "pir": spin[-1],
        "erdi": spin[0] - spin[1] - 50,
        "spdi": spin[0] - spin[-2] - 50,
    }
).plot.line(x="Freq", y=["lw", "er", "on", "sp", "pir", "erdi", "spdi"])
fig.update_xaxes(type="log")
fig.show()

In [ ]:
lw_diff = spin[0] - python_spin["Listening Window"]
er_diff = spin[1] - python_spin["Early Reflections"]
sp_diff = spin[-2] - python_spin["Sound Power"]
pir_diff = spin[-1] - python_pir["Estimated In-Room Response"]
df_diff = pd.DataFrame(
    {
        "Freq": freq,
        "lw": lw_diff,
        "er": er_diff,
        "sp": sp_diff,
        "pir": pir_diff,
    }
)
diff = df_diff.plot.line(x="Freq", y=["lw", "er", "sp", "pir"])
diff.update_xaxes(type="log")
diff.update_yaxes(range=[-1, 1])
diff.show()

In [ ]:
%timeit c_score(freq.values, intervals, spl_h['On Axis'].values, spin[0], spin[-2], spin[-1])

In [ ]:
c_score(freq.values, intervals, spl_h["On Axis"].values, spin[0], spin[-2], spin[-1])

In [ ]:
from spinorama.load_rewseq import parse_eq_iir_rews
from spinorama.filter_peq import peq_build

peq = parse_eq_iir_rews(f"../datas/eq/{speaker_name}/iir.txt", 48000)
peq_spl = peq_build(freq, peq)

In [ ]:
spin_peq, score_peq = c_score_peq(
    freq.values,
    idx_cea2034,
    intervals,
    np.array(weigths),
    spl_h.T.to_numpy(),
    spl_v.T.to_numpy(),
    peq_spl,
)

In [ ]:
score_peq

In [ ]:
fig = pd.DataFrame(
    {
        "Freq": freq,
        "lw": spin_peq[0],
        "er": spin_peq[1],
        "sp": spin_peq[-2],
        "pir": spin_peq[-1],
        "erdi": spin_peq[0] - spin_peq[1] - 50,
        "spdi": spin_peq[0] - spin_peq[-2] - 50,
    }
).plot.line(x="Freq", y=["lw", "er", "sp", "pir", "erdi", "spdi"])
fig.update_xaxes(type="log")
fig.show()